# Refinitiv Data Library for Python
## Content - Pricing - Synchronous calls

This notebook demonstrates how to use the Pricing interface to retrieve snapshot data from the Refinitiv Data Platform.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
# Open the Default Session
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x21234aa8c70 {session_name='rdp'}>

## Get Price Snapshot
The following gets a snapshot from the non-streaming RDP Pricing API - note the **get_data()** call

In [4]:
# Define our RDP Snapshot Price object
response = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

if (response.is_success):
    display(response.data.df)
else:
    print(response.http_status)

,Instruments,BID,ASK
0,EUR=,1.1218,1.1221
1,GBP=,1.332,1.3321
2,JPY=,115.31,115.34
3,CAD=,1.2661,1.2665


## Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a **Deployed** session.   
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed (assuming you have a Streaming Data licence) - note the **get_stream()** call

In [5]:
# Define our Streaming Price object
non_streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# We want to just snap the current prices, don't need updates
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)
# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

,Instrument,ASK,BID
0,EUR=,1.1221,1.122
1,GBP=,1.3323,1.3318
2,JPY=,115.33,115.32
3,CAD=,1.2662,1.2658


The non-streaming requests are closed by the server - **no need to close them**.   

### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [6]:
non_streaming['EUR=']['BID']

1.122

In [7]:
eur = non_streaming['EUR=']
eur['ASK']

1.1221

#### Iterate on fields

In [8]:
print('JPY=')
for field_name, field_value in non_streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 115.32
	ASK : 115.33


#### Iterate on instruments and fields

In [9]:
for instrument in non_streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.122
	ASK : 1.1221
GBP=
	BID : 1.3318
	ASK : 1.3323
JPY=
	BID : 115.32
	ASK : 115.33
CAD=
	BID : 1.2658
	ASK : 1.2662


## Close the session

In [10]:
rd.close_session()